In [1]:
 %pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
from pylab import *
from scipy import *
from scipy import stats, io
import numpy as np
import struct
import tables as tb
from attrdict import AttrDict
import matplotlib.pyplot as plt
import os as os
from phy.io import KwikModel
import codecs as codecs
import matplotlib.gridspec as gridspec
import matplotlib.ticker as mtick

# FUNCTIONS TO BUILD PSTHs STAs AND STCs

In [27]:
#----------------------------------------------------------------------------------------
# READ STIMULUS
#----------------------------------------------------------------------------------------
# Here we read the binary file with stimulus: 902 of 25 piezos x 1024 samples
# Text file has the type of stimulus: F sparse, C correlated, U uncorrelated
def read_stimulus(expe,meas,lenstim):
    read_data = []
    copy_data = []
    txt_data = []
    
    global UFC
    UFC = np.zeros(3)
    
    if expe>22: length = 923
    else: length = 902   #stim files are different since exp 23
    
    #we load the stim file
    bin_file = open(binname,'rb')
    read_data = np.fromfile(file=bin_file, dtype=np.float32)
    read_data = read_data.reshape((length,25,10240))
    txt_data = np.loadtxt(textname, dtype='S8')
    txt_data = txt_data.view(np.chararray).decode('utf-8')
    
    #lenstim comes from the Vtag file for experiments that were ended before the last stim
    if expe>22:
        length1= lenstim
        length2=0
        
        txt_data = np.append(txt_data[0:length1], txt_data[0:length2])

        read_data = np.append(read_data[0:length1], read_data[0:length2],0)
        read_data = read_data.reshape((length1+length2,25,10240))
        close(binname)

        bin_file = open(binname,'rb')
        copy_data = np.fromfile(file=bin_file, dtype=np.float32)
        copy_data = copy_data.reshape((length,25,10240))

        copy_data = np.append(copy_data[0:length1], copy_data[0:length2],0)
        copy_data = copy_data.reshape((length1+length2,25,10240))
        close(binname)
        
        UFC = np.zeros(3)
        for i in range(lenstim):
            if txt_data[i] == 'U': UFC[0]+=1
            elif txt_data[i] == 'F': UFC[1]+=1
            elif txt_data[i] == 'C': UFC[2]+=1
        
    else:
        #-----------------------------------------------------------------------------------------------------------
        #here we treat each experiment differently since we stopped before end, and recorded more than once in a site
        if expe == 21 and meas ==2:
            length2 =357
            extra = 63
            #there are 63 extra events in exp2 and 358 in exp3
            ext = [0]*extra*25*10240
            ext=np.array(ext)
            ext = ext.reshape(extra,25,10240)
            ext_text = ['B']*extra

            txt_data = np.append(txt_data[0:length],ext_text)
            txt_data = np.append(txt_data, txt_data[0:length2])
            read_data = np.append(read_data[0:length],ext,0)
            read_data = np.append(read_data,read_data[0:length2],0)   

            close(binname)

            bin_file = open(binname,'rb')
            copy_data = np.fromfile(file=bin_file, dtype=np.float32)
            copy_data = copy_data.reshape((length,25,10240))
            copy_data = np.append(copy_data[0:length],ext,0)
            copy_data = np.append(copy_data,copy_data[0:length2],0)   

            close(binname)
        else:
            if expe == 18 or (expe ==20 and meas==1) or (expe==21 and meas==1) or (expe==22 and meas==2):
                    length1=902
                    length2 =0
            if expe == 20 and meas==2:
                    length1 = 46 
                    #there are 46 events in exp2 and 879 in exp3
                    length2 =879
            if expe == 22 and meas==1:
                    length1=902
                    length2 =93

            txt_data = np.append(txt_data[0:length1], txt_data[0:length2])

            read_data = np.append(read_data[0:length1], read_data[0:length2],0)
            read_data = read_data.reshape((length1+length2,25,10240))
            close(binname)

            bin_file = open(binname,'rb')
            copy_data = np.fromfile(file=bin_file, dtype=np.float32)
            copy_data = copy_data.reshape((length,25,10240))

            copy_data = np.append(copy_data[0:length1], copy_data[0:length2],0)
            copy_data = copy_data.reshape((length1+length2,25,10240))
            close(binname)

        #this correction applies until exp 22, then whisker assignment is fine    
        read_data[:, 11, :] = copy_data[:, 24, :]
        read_data[:, 20, :] = copy_data[:, 11, :]
        read_data[:, 21, :] = copy_data[:, 20, :]
        read_data[:, 22, :] = copy_data[:, 21, :]
        read_data[:, 23, :] = copy_data[:, 22, :]
        read_data[:, 24, :] = copy_data[:, 23, :]
        
    
        UFC = np.zeros(3)
        for i in range(len(read_data)):
            if txt_data[i] == 'U': UFC[0]+=1
            elif txt_data[i] == 'F': UFC[1]+=1
            elif txt_data[i] == 'C': UFC[2]+=1
       #-----------------------------------------------------------------------------------------------------------
    
    return read_data, txt_data
#----------------------------------------------------------------------------------------
# READKWIKINFO
#----------------------------------------------------------------------------------------
# We read the data of the output from klusterkwik: spike times and cluster-number of each
# cluster-number is in klustaviewa series (can be as high as 130 e.g.)
def readkwikinfo(kwik, grupete):
    model = KwikModel(kwik)
    spiketimes = model.spike_times
    clusters = model.cluster_groups
    sample_rate = model.sample_rate
    
    spikedata = dict()
    for cluster in clusters.keys():
        clustergroup = clusters[cluster]
                
        if clustergroup==grupete:
            spiketimematrix = AttrDict({'spike_times': np.zeros(len(spiketimes[where(model.spike_clusters==cluster)]))})
            spiketimematrix.spike_times = spiketimes[where(model.spike_clusters==cluster)]
            spikedata[cluster] = spiketimematrix
    
    return spikedata, sample_rate
#----------------------------------------------------------------------------------------
# BUILDS PSTH
#----------------------------------------------------------------------------------------
def BuildPSTH(Vtag1,stim,stimtype, Spikes, sampling_freq, t_before, t_after,starts,stops) :
## The first task is to find the stimulus onset times for each whisker in each sweep in each direction
    #stim, stimtype = read_stimulus()
    stim = stim[np.where(stimtype=='F')[0], :, :]

    stimtimes = {}
    stimspersweep = 6  #as the maximum
    FCsweeps = UFC[1]
       
    for w in np.arange(25, dtype='int') :
        timesUP = np.zeros((FCsweeps, stimspersweep))
        timesDOWN = np.zeros((FCsweeps, stimspersweep))
        for i in np.arange(len(stim), dtype='int') :
            indsUP = np.where(stim[i, w, :]>0)[0]-1   #we correct for 0 at the start of stim
            diffUP = indsUP[1:] - indsUP[:-1]
            stimsup = len(indsUP)/28
            timesUP[i,0:stimsup] = np.insert(indsUP[np.where(diffUP>1)[0]+1], 0, indsUP[0])
            
            indsDOWN = np.where(stim[i, w, :]<0)[0]-1 #we correct for 0 at the start of stim
            diffDOWN = indsDOWN[1:] - indsDOWN[:-1]
            stimsdown = len(indsDOWN)/28
            timesDOWN[i,0:stimsdown] = np.insert(indsDOWN[np.where(diffDOWN>1)[0]+1], 0, indsDOWN[0])
            
        stimtimes[w] = timesUP, timesDOWN #stimtimes[whisker][0]=UP stimtimes[whisker][1]=DOWN
    
    # make an 'output dict'
    # the PSTH will be built on -tbefore:tafter
    hist_inds = {}
    PSTH_spike_counts = {}
    
    ## Now we will take the stim times and find all the relevant spikes
    #start_and_stops = Vtag1[1:] - Vtag1[:-1]
    #starts = (where(start_and_stops==1)[0]-2999)/float(sampling_freq) # time in seconds
    #stops = (where(start_and_stops==-1)[0]+4110)/float(sampling_freq) # time in seconds
    starts = starts[np.where(stimtype=='F')[0]]
    stops = stops[np.where(stimtype=='F')[0]]
    
    # Loop each neuron and get the spikes.
    for neuron in Spikes.keys(): 
        PSTH_spike_counts[neuron], hist_inds[neuron] = PSTH_spikes(stim, stimtype, stimtimes, Spikes[neuron].spike_times, sampling_freq, t_before, t_after, starts, stops)
    return PSTH_spike_counts, hist_inds

#----------------------------------------------------------------------------------------

def PSTH_spikes(stimulation, stimtype, stimtimes, spikes, samp, t_before, t_after, starts, stops):
    """
    stimulation   : a list of numpy arrays with a n*t stimulus inside
    stimtimes     : a list of the times the stimulus occurred for each whisker 
    spikes        : an array that contains the spike times (s)
    Vtag1         : synchronises stimulus with spike times
    samp          : sampling rate of the stimulation (Hz)
    t_before      : duration before the stim (positive, s)
    t_after       : duration after the stim (positive, s)
    starts        : the start of the F sweeps
    stops         : the stops of the F sweeps
    """
    
    stim_samp = 1/.0009997575757 
    PSTH_spike_counts = {}
    
    # Loop all whiskers for the spikes following the stimuli
    for w in np.arange(25, dtype='int') :
        spikecountsup = 0
        spikecountsdown = 0
        for i in np.arange(len(stimulation), dtype='int') : 
            for x in np.arange(6, dtype='int') :           
                if (stimtimes[w][0][i, x]>0) & (stimtimes[w][1][i, x]>0) :
                    timesUP = starts[i] + stimtimes[w][0][i, x]/stim_samp
                    timesDOWN = starts[i] + stimtimes[w][1][i, x]/stim_samp

                    ## select spikes which fall near this stimulus
                    spikecountsup += len(spikes[(timesUP - t_before < spikes) * (spikes < timesUP + t_after)])
                    spikecountsdown += len(spikes[(timesDOWN - t_before < spikes) * (spikes < timesDOWN + t_after)])
        PSTH_spike_counts[w] = spikecountsup, spikecountsdown
    
    hist_inds = {}
    
    for w in np.arange(25, dtype='int') :
        hist_inds[w] = np.zeros(PSTH_spike_counts[w][0]), np.zeros(PSTH_spike_counts[w][1])
        spikecountsup = 0
        spikecountsdown = 0
        for i in np.arange(len(stimulation), dtype='int') : 
            for x in np.arange(6, dtype='int') :           
                if (stimtimes[w][0][i, x]>0) & (stimtimes[w][1][i, x]>0) : #to rule out cases with less than 5 stims per sweep
                    timesUP = starts[i] + stimtimes[w][0][i, x]/stim_samp
                    timesDOWN = starts[i] + stimtimes[w][1][i, x]/stim_samp

                    ## select spikes which fall near this stimulus
                    spikecountup = len(spikes[(timesUP - t_before < spikes) * (spikes < timesUP + t_after)])
                    spikecountdown = len(spikes[(timesDOWN - t_before < spikes) * (spikes < timesDOWN + t_after)])

                    spikeidxup = spikes[(timesUP - t_before < spikes) * (spikes < timesUP + t_after)]
                    spikeidxdown = spikes[(timesDOWN - t_before < spikes) * (spikes < timesDOWN + t_after)]
                    spikeidxup = np.around((spikeidxup - starts[i])/float(stops[i] - starts[i])*len(stimulation[i,0]))
                    spikeidxdown = np.around((spikeidxdown - starts[i])/float(stops[i] - starts[i])*len(stimulation[i,0]))

                    hist_inds[w][0][spikecountsup:(spikecountsup+spikecountup)] = spikeidxup-stimtimes[w][0][i, x]
                    hist_inds[w][1][spikecountsdown:(spikecountsdown+spikecountdown)] = spikeidxdown-stimtimes[w][1][i, x]

                    spikecountsup += spikecountup
                    spikecountsdown += spikecountdown
    return PSTH_spike_counts, hist_inds

#----------------------------------------------------------------------------------------

def GetWhiskers(histdata, t_before, t_after, thresh,activity,chancelevel) :
    STC_on = {}
    PW = {}
    ActMod2={}
    ActModout={}
    
    
    stim_samp = 1/.0009997575757 
    before_index = int(np.around(t_before*stim_samp)) 
    after_index = int(np.around(t_after*stim_samp)) 
    bins = before_index + after_index
    
    for neuron in histdata.keys() :
        ActMod = np.zeros(25)
        Countinout = np.zeros(25)
        Count=np.zeros(25)
        SHARP = np.zeros(25)
        for i in np.arange(25, dtype='int') :
            after = 0
            before = 0
            countin = 0
            countout =0
            
            for j in np.arange(len(histdata[neuron][i][0]), dtype='int') :
                if histdata[neuron][i][0][j]>9 and histdata[neuron][i][0][j]<45 : 
                    after+=1                              #conunt spikes in 30ms timewindow after stimulus
                    countin += 1
                elif histdata[neuron][i][0][j]<10 :                
                    before+=1                             #conunt spikes in 20ms timewindow befor stimulus
                    countout += 1
            for j in np.arange(len(histdata[neuron][i][1]), dtype='int') :
                if histdata[neuron][i][1][j]>9 and histdata[neuron][i][1][j]<45 :
                    after+=1                              
                    countin += 1
                elif histdata[neuron][i][1][j]<10:
                    before+=1
                    countout += 1
            if (after+before)==0 :
                before=1
            elif (after+before)>activity : 
                ActMod[i]= (  after*(before_index+10) - before*35  ) / (  after*(before_index+10) + before*35  )   #weight different time windows
                Countinout[i] = countin/(countout+1)
                condition1 = where((histdata[neuron][i][0]<20)*(histdata[neuron][i][0]>5))
                condition2 = where((histdata[neuron][i][1]<20)*(histdata[neuron][i][1]>5))
                SHARP[i] = sum( histdata[neuron][i][0][ condition1 ] ) + sum( histdata[neuron][i][1][condition2] )
            else :
                ActMod[i]=0               
            Count[i]=countin+countout
            
        ActModout[neuron]=ActMod
        ActMod2[neuron]=ActMod*Countinout*SHARP
        STC_on[neuron] = (ActMod>thresh)*(Count/sum(Count)>chancelevel)
        PW[neuron] = np.where(ActMod2[neuron]==max(ActMod2[neuron]))[0][0]
        if size(np.where(STC_on[neuron]==True))==0:
            PW[neuron]= 20
        
        
    return STC_on,PW, ActModout,ActMod2

#----------------------------------------------------------------------------------------
# BUILDS LATENCIES
#----------------------------------------------------------------------------------------
def BuildLatencies(histdata,STC_on,t_before, t_after,zth,lookbins) :
    
    stim_samp = 1/.0009997575757 
    before_index = int(np.around(t_before*stim_samp)) 
    after_index = int(np.around(t_after*stim_samp)) 
    nbins = before_index + after_index
    bins=np.linspace(-before_index, after_index, nbins+1)   #here we use 50 bins, so linspace has the bin delimiters
                                                            # first bin between -5 and -4, and last 44 and 45 for example  
    Latencies = {}
    LatMean ={}
    ResponseRC = {}
    Zsc={}
    
    for neuron in sorted(histdata.keys())[:] :
        
        resp = zeros(25)
        lat = zeros([2,25])
        latM = zeros([2,25])
        Zsc[0]=zeros([25,nbins])
        Zsc[1]=zeros([25,nbins])
        
        histo = np.append(np.histogram(hist_output[neuron][20][0],bins=bins)[0],np.histogram(hist_output[neuron][20][1],bins=bins)[0])
        mu = np.mean(histo) 
        if mu<0.1: mu = 0.75
        sigma = np.std(histo)
               
        if len(np.where(STC_on[neuron]==True)[0])>0:         #if list not empty
            for i in np.where(STC_on[neuron]==True)[0] : 
                
                #compute zscore for each bin of each whisker that is active
                Zsc[0][i] = (np.histogram(histdata[neuron][i][0],bins=bins)[0]-mu)/sigma
                Zsc[1][i] = (np.histogram(histdata[neuron][i][1],bins=bins)[0]-mu)/sigma
                
                #----------------------------------------------------------------------------------------
                #Find latencies for positive stimulus
                #compute Max and sums in area of interest                
                Max = np.where(Zsc[0][i][before_index+5:-10]==max(Zsc[0][i][before_index+5:-10]))[0][0]  + before_index+5      
                max0 = max(Zsc[0][i])
                sum0 = sum(Zsc[0][i][before_index+10:before_index+45])
                
                #get rid of whiskers without enough response, looking around the max bin
                if Zsc[0][i][Max]<5 and sum(Zsc[0][i][Max-4:Max+1])/5 < 1.5:
                    lat[0][i] = float('nan')
                else:
                    zth2=zth
                    lookb = lookbins
                    #if max zscore is not high enough, increase zthreshold and decrease looking window size
                    if max0<12: 
                        zth2 = zth2*1.5
                        lookb=lookb-1
                    
                    #here we loop until finding the start bin of the response
                    should_restart = True
                    while should_restart:
                        should_restart = False
                        for j in arange(Max)[-lookb::-1]:  
                            #look back from the max_bin in a lookb window averaging below zth or a until a big slope
                            if (sum(Zsc[0][i][j:j+lookb])/lookb<zth2) or ( (Zsc[0][i][j+lookb]-sum(Zsc[0][i][j:j+lookb-1]))/(lookbins-1)>zth2*1.):  
                                #this shifts accounts for the window size
                                shift = np.where(Zsc[0][i][j:j+lookb+1]>zth2)[0][0]
                                #bins are centered at +.5, from -before_index, so we add 0.5
                                #and another 1 since we got here most probably when max is out of the searhc window
                                lat[0][i] = j-before_index+ 1.5 +shift*0.75   
                                latM[0][i] = (Max-before_index+lat[0][i])/2
                                #loop restarts if latency is below 5msec with higher zthreshold
                                if lat[0][i]<5:        
                                    should_restart =True       
                                    zth2 = zth2*1.2
                                break                           
                #----------------------------------------------------------------------------------------
                #We repeat for the negative stimulus
                #compute Max and sums in area of interest                
                Max = np.where(Zsc[1][i][before_index+5:-10]==max(Zsc[1][i][before_index+5:-10]))[0][0]  + before_index+5      
                max1 = max(Zsc[1][i])
                sum1 = sum(Zsc[1][i][before_index+10:before_index+45])
                
                #get rid of whiskers without enough response, looking around the max bin
                if Zsc[1][i][Max]<5 and sum(Zsc[1][i][Max-4:Max+1])/5 < 1.5:
                    lat[1][i] = float('nan')
                else:
                    zth2=zth    
                    lookb = lookbins
                    #if max zscore is not high enough, increase zthreshold and decrease looking window size
                    if max1<12: 
                        zth2 = zth2*1.5
                        lookb=lookb-1

                    #here we loop until finding the start bin of the response
                    should_restart = True
                    while should_restart:
                        should_restart = False
                        for j in arange(Max)[-lookb::-1]:
                            #look back from the max_bin in a lookb window averaging below zth or a until a big slope
                            if sum(Zsc[1][i][j:j+lookb])/lookb<zth2 or ( (Zsc[1][i][j+lookb]-sum(Zsc[1][i][j:j+lookb-1]))/(lookb-1)>zth2*1.):
                                #this shifts accounts for the window size
                                shift = np.where(Zsc[1][i][j:j+lookb+1]>zth2)[0][0]
                                lat[1][i] = j-before_index+ 1.5 +shift*0.75
                                latM[1][i] = (Max-before_index+lat[1][i])/2
                                #loop restarts if latency is below 5msec with higher zthreshold
                                if lat[1][i]<5: 
                                    should_restart =True
                                    zth2 = zth2*1.2
                                break
                #----------------------------------------------------------------------------------------
                #Compute the direction of the dominant response 1 positive -1 negative
                if max0+sum0/35>max1+sum1/35: resp[i]=1
                else : resp[i]=-1
            #----------------------------------------------------------------------------------------                
            #Output variables
            Latencies[neuron] = lat
            LatMean[neuron] = latM
            ResponseRC[neuron]= resp
       
    return Latencies, LatMean , ResponseRC

#----------------------------------------------------------------------------------------
# BUILD SPIKE TABLES
#----------------------------------------------------------------------------------------
# Code to guide construction of the spike tables with rows representing variables (whisker and time) and each spike has a column        
def BuildSpikeTables(Spikes, Vtag1,stim,stimtype, sampling_freq, t_before, t_after, choice,starts,stops):
    
    #start_and_stops = Vtag1[1:] - Vtag1[:-1]
    #starts = (np.where(start_and_stops==1)[0]-2999)/float(sampling_freq)
    #stops = (np.where(start_and_stops==-1)[0]+4110)/float(sampling_freq)
    
    #if len(starts)>len(stops): starts = [starts, stops[-1]+0.01]
    
    Spike_counts = {}
    Spike_Tables = {}
    
    #stim, stimtype = read_stimulus()
    
    for neuron in Spikes.keys():
        Spike_counts[neuron], Spike_Tables[neuron] = GenSpikeTables(stim, Spikes[neuron].spike_times, starts, stops, sampling_freq, t_before, t_after, stimtype, choice)
    return Spike_counts, Spike_Tables


#----------------------------------------------------------------------------------------
# GEN SPIKE TALBLES
#----------------------------------------------------------------------------------------
# This generates the spike tables with rows representing variables (whisker and time) and each spike has a column
def GenSpikeTables(stimulation, spikes, t_start, t_stop, samp, t_before, t_after, stimtype, choice) :
    """
    stimulation : a numpy array with a n*t stimulus inside
    spikes      : an array that contains the spike times (s)
    t_start     : the stimulation start time (s)
    samp        : sampling rate of the stimulation (Hz)
    t_before    : duration before the stim (positive, s)
    stimtype    : a vector of the type of stimulation, 'U' = uncorrelated, 'C' = correlated; 'F' = forward
    choice      : what kind of stimulation to analyse
    """

    stim_samp = 1/.0009997575757
    
    before_index = np.int(np.around(t_before*stim_samp))
    after_index = np.int(np.around(t_after*stim_samp))
    iterator = before_index + after_index

    spike_count = 0

    for i in np.arange(len(stimulation), dtype='int'):
        if stimtype[i]==choice :
            ## select spikes who fall in this particular sweep
            spikeidx = spikes[( (t_start[i]+t_before) < spikes) * (spikes < (t_stop[i]-t_after))]
            spike_count += len(spikeidx)
    
    spike_table = np.zeros((iterator*25, spike_count), dtype='int')
    spikenumber = 0
    for i in np.arange(len(stimulation), dtype='int'):
        if stimtype[i]==choice :
            ## select spikes who fall in this particular sweep
            spikeidx = spikes[( (t_start[i]+t_before) < spikes) * (spikes < (t_stop[i]-t_after))]
            ## calculate the index in the sweep that the spike occurred
            spikeidx = np.around((spikeidx - t_start[i])/float(t_stop[i] - t_start[i])*len(stimulation[i,0]))    
            for s in spikeidx:
                spikewindow = stimulation[i, :, np.int(s)-before_index:np.int(s)+after_index]
                spike_table[:, spikenumber] = np.reshape(spikewindow, [size(spikewindow)])
                spikenumber = spikenumber + 1
    return spike_count, spike_table


#----------------------------------------------------------------------------------------
# BUILD STA
#----------------------------------------------------------------------------------------
# Build STAs from spike tables, just averages across rows of the spike table
def Build_STA(Spike_Tables) :
    STA_Tables = {}

    for neuron in Spike_Tables.keys():
        STAraw = np.mean(Spike_Tables[neuron],axis=1)
        STA_Tables[neuron] = np.reshape(STAraw,(25,len(STAraw)/25))
    
    return STA_Tables

#----------------------------------------------------------------------------------------
# BUILD STC
#----------------------------------------------------------------------------------------
# Build STCs from spike tables
def Build_STC(Spike_Tables,stim,stimtype, whiskers, choice, numsamples, t_before, t_after, stimtreatment, cutoff) :
    ## First we randomly select parts of the stimulus and compute the covariance matrix for the stimulus.
    
    stim_samp = 1/.0009997575757
    
    before_index = np.int(np.around(t_before*stim_samp))
    after_index = np.int(np.around(t_after*stim_samp))
    iterator = before_index+after_index

    #stim, stimtype = read_stimulus()
    inds = np.where(stimtype==choice)[0]
    stimtrim = stim[inds,:,110:10145]         #we are only using the stimulus part of 'C' or 'U'
    
    Nstimcovmat = {}
    W = {}
    
    if stimtreatment=='whiten' :
        for neuron in Spike_Tables.keys() :
            whiskinds = np.where(whiskers[neuron]==True)[0]
            if np.where(whiskers[neuron]==True)[0].size==0 :
                #whiskinds = np.arange(25, dtype='int')    #uncomment to analyse all whiskers when there is no significant one
                whiskinds = np.array([20])                   #uncomment to speed up and not evaluate neurons without significant whiskers
            spike_table = np.zeros((len(whiskinds)*iterator, numsamples))

            for sample in np.arange(numsamples, dtype='int') :
                stimsweep = randint(0, stimtrim.shape[0])
                s = randint(before_index, stimtrim.shape[2]-after_index)
                spikewindow = stimtrim[stimsweep, whiskinds, s-before_index:s+after_index]
                spike_table[:,sample] = np.reshape(spikewindow, [size(spikewindow)])
        
            #spike_table = np.mean(np.reshape(spike_table, [2, -1, numsamples], 'F'), axis=0)
            indsdown = np.zeros((len(whiskinds),np.int(((iterator/5)+1))), dtype='int')
            k=1
            for i in np.arange(((iterator/5)+1), dtype='int') :
                if i>0 :
                    k=0
                for j in np.arange(len(whiskinds), dtype='int') :
                    indsdown[j,i] = np.int(i*5+j*50+k-1)
            indsdown[0, 0] = 0
            indsdown = np.reshape(indsdown, [size(indsdown)])

            spike_table = spike_table[indsdown, :]
        
            spike_table = spike_table - np.mean(spike_table, axis=1)[:, np.newaxis] # mean centering
            Nstimcovmat[neuron] = np.cov(spike_table)
            eigval1, eigvec1 = np.linalg.eigh(Nstimcovmat[neuron])
    
            inds = np.argsort(eigval1)[::-1]
            eigval = eigval1[inds]
            eigvec = eigvec1[:, inds]
    
            scalede = np.sqrt(np.real(eigval))
            inveigval = np.diag(1/scalede)
    
            n_eigvecs = np.int(np.around(len(eigvec)*cutoff))
            inveigval[n_eigvecs:] = 0

            W[neuron] = np.dot(eigvec, np.dot(inveigval, eigvec.T))
    
    #Second we must whiten the spike triggered ensemble, and do an eigenvalue decomposition of the covariance matrix
    #for each neuron 
    STC_covmats = {}
    STC_eigs = {}
    
    for neuron in Spike_Tables.keys() :
        #For STC we usually will not take the full 100ms before and after the stimulus
        #These lines trim the stimulus down 
        whiskinds = np.where(whiskers[neuron]==True)[0]
        if np.where(whiskers[neuron]==True)[0].size==0 :
            #whiskinds = np.arange(25, dtype='int')    #uncomment to analyse all whiskers when there is no significant one
            whiskinds = np.array([20])                   #uncomment to speed up and not evaluate neurons without significant whiskers

        spike_table = Spike_Tables[neuron]
        spike_table = np.reshape(spike_table, [spike_table.shape[0]/25, 25, spike_table.shape[1]], 'F')

        
        spike_table = spike_table[:, whiskinds, :]
        spike_table = np.reshape(spike_table, [spike_table.shape[0]*spike_table.shape[1], spike_table.shape[2]], 'F')
        
        
        indsdown = np.zeros((len(whiskinds),np.int(((iterator/5)+1))), dtype='int')
        k=1
        for i in np.arange(((iterator/5)+1), dtype='int') :
            if i>0 :
                k=0
            for j in np.arange(len(whiskinds), dtype='int') :
                indsdown[j,i] = np.int(i*5+j*50+k-1)
        indsdown[0, 0] = 0
        indsdown = np.reshape(indsdown, [size(indsdown)])
        spike_table = spike_table[indsdown, :]

        #spike_table = np.mean(np.reshape(spike_table, [2, -1, spike_table.shape[1]], 'F'), axis=0)
        spike_table = spike_table - np.mean(spike_table,axis=1)[:, np.newaxis] # mean centering
        
        
        if stimtreatment == 'remcorr' :
            STC_covmats[neuron] = np.cov(np.dot(eigvec1.T, spike_table)) #with spatial correlation removed
        elif stimtreatment == 'whiten' :
            STC_covmats[neuron] = np.cov(np.dot(W[neuron], spike_table)) #with whitening
        elif stimtreatment == 'none':
            STC_covmats[neuron] = np.cov(spike_table) #raw spike triggered ensemble
            
        STCeig = AttrDict({'eigvals': np.zeros(STC_covmats[neuron].shape[0]), 'eigvecs': np.zeros((STC_covmats[neuron].shape[0], STC_covmats[neuron].shape[0]))})
        STCeig.eigvals, STCeig.eigvecs = np.linalg.eigh(STC_covmats[neuron])
        STC_eigs[neuron] = STCeig
    
    return STC_eigs, STC_covmats, W, Nstimcovmat

# DISPLAYING FUNCTIONS

In [4]:
#----------------------------------------------------------------------------------------
# DISPLAY PSTH 
#----------------------------------------------------------------------------------------
# Plot a single neuron PSTH, 25 piezos
def display_PSTH(histdata, counts, t_before, t_after,fig,inner_grid,neuron,numspikesP,numspikesN,STC_on,PW) :
    stim_samp = 1/.0009997575757 
    before_index = int(np.around(t_before*stim_samp)) # indexes
    after_index = int(np.around(t_after*stim_samp)) # indexes
    histlength = before_index + after_index + 1
    
    nup = np.zeros((25,histlength-1))
    ndown = np.zeros((25,histlength-1))
    
    
    fig2 = figure()
    ax = fig2.add_subplot(1,1,1)
    for i in range(25) :
        if histdata[i][0].size :
            n1, bins, patches = ax.hist(histdata[i][0], bins = np.linspace(-before_index, after_index, histlength))
            nup[i,:] = n1
            close()
        if histdata[i][1].size :
            n2=2
            n2, bins, patches = ax.hist(histdata[i][1], bins = np.linspace(-before_index, after_index, histlength))
            ndown[i,:] = n2
            close()
    normnum = (1/np.sum(nup+ndown))
    height = np.max(np.array([np.max(nup), np.max(ndown)]))/(1/normnum)
     
   
        
    clf()
    
    for i in range(25) :
        if i == 0 :
            #ax1 = subplot(5,5,1, frame_on=False)
            ax1 = Subplot(fig, inner_grid[i])     
            ax1.set_xticks([])
            ax1.set_yticks([])
        elif i==20:
            ax1 = Subplot(fig,inner_grid[i],sharex=ax1,sharey=ax1)
            ax1.spines['right'].set_linewidth(0.3)
            ax1.spines['top'].set_linewidth(0.3)
            ax1.spines['left'].set_linewidth(0.3)
            ax1.spines['bottom'].set_linewidth(0.3)
            
            ax1.set_xticks([])
            ax1.set_yticks([])
        else :
            #subplot(5,5,i+1,sharex=ax1,sharey=ax1,frame_on=False)
            ax1 = Subplot(fig,inner_grid[i],sharex=ax1,sharey=ax1)
            ax1.set_xticks([])
            ax1.set_yticks([])
        
        if PW==i:
            ax1.set_axis_bgcolor('#dddddd')    
        elif i!=20:
            ax1.spines['right'].set_visible(False)
            ax1.spines['top'].set_visible(False)
            ax1.spines['left'].set_visible(False)
            ax1.spines['bottom'].set_visible(False)
        if STC_on[i]== True:
            ax1.set_axis_bgcolor('#dddddd')    
                        
        
        if histdata[i][1].size :
            ax1.hist(histdata[i][1], bins = np.linspace(-before_index, after_index, histlength), color='g', alpha=1.0, edgecolor='none', histtype='stepfilled', label='Pos', weights=np.repeat(normnum, len(histdata[i][1])))
        if histdata[i][0].size :
            ax1.hist(histdata[i][0], bins = np.linspace(-before_index, after_index, histlength), color='b', alpha=0.7, edgecolor='none', histtype='stepfilled', label='Neg', weights=np.repeat(normnum, len(histdata[i][0]))) 
        #if (histdata[i][0].size) or (histdata[i][1].size) :
        xlim(-before_index, after_index)
        ax1.axvline(0, color = 'r', linewidth=1)
        ax1.axhline(0, color = 'r', linewidth=2)
        ymax = 1.02 * height
        ylim(0, ymax)
        xvals = np.array([0,10,20,30])
        yvals = np.array([0,ymax*0.9,ymax*0.9,0])
        ax1.plot(xvals, yvals, linewidth=0.2,color = (0.75,0.75,0.75))
        if i==4: ax1.set_title('ymax =' + str( np.around(height,decimals = 3) ),fontsize=8)
        if i ==1: ax1.set_title('Nrn' + str(neuron) + '_Pos' + str(int(numspikesP))+ '_Neg' + str(int(numspikesN)),fontsize=9)

        
        fig.add_subplot(ax1)

#----------------------------------------------------------------------------------------
def display_all_PSTHs_of_recording(histdata, counts, pdf_files_directory, t_before, t_after,grupete,STC_on,PW,titles) :
    
    fig = plt.figure(figsize=(12,16.5))
    nrns = len(histdata.keys())
    if nrns <16: 
        layout = [5,3]
    else: layout = [nrns//3+(nrns%3!=0),3]
    outer_grid = gridspec.GridSpec(layout[0], layout[1], wspace=0.1, hspace=0.2)
    
    ii=0
    
    orderneurons = np.sort(list(histdata.keys()))
    for neuron in  orderneurons:
        clf()
        totalup = 0
        totaldown = 0
        for i in np.arange(25, dtype='int') :
            totalup+=counts[neuron][i][0]
            totaldown+=counts[neuron][i][1]
            
        inner_grid = gridspec.GridSpecFromSubplotSpec(5,5,subplot_spec=outer_grid[ii], wspace=0.1, hspace=0.1)
               
        numspikesP= totalup                  
        numspikesN= totaldown
        display_PSTH(histdata[neuron], counts[neuron], t_before, t_after,fig,inner_grid,neuron,numspikesP,numspikesN,STC_on[neuron],PW[neuron])                               
        
        if grupete ==1:
            fig.suptitle(titles + '_multiunits',fontsize=16)
        else:
            fig.suptitle(titles ,fontsize=16)
        
        ii+=1
    if grupete ==1:                  
        fig.savefig(pdf_files_directory + titles + '_hist_multi.pdf', format='pdf')
    else:
        fig.savefig(pdf_files_directory + titles + '_hist.pdf', format='pdf')
        
    clf()

#----------------------------------------------------------------------------------------
#----------------------------------------------------------------------------------------          
#----------------------------------------------------------------------------------------
# DISPLAY LATENCIES
#----------------------------------------------------------------------------------------
# Plot a single neuron PSTH, 25 piezos
def display_Latencies(PW, STC_on, ResRC, Latencies,titles,grupete,pdf_files_directory) :
    
    AdGraph = {}

    AdGraph[0] = array([1,5,6])
    AdGraph[1] = array([0,2,5,6,7])
    AdGraph[2] = array([1,3,6,7,8])
    AdGraph[3] = array([2,4,7,8,9])
    AdGraph[4] = array([3,8,9])

    AdGraph[5] = array([0,1,6,10,11])
    AdGraph[6] = array([0,1,2,5,7,10,11,12])
    AdGraph[7] = array([1,2,3,6,8,11,12,13])
    AdGraph[8] = array([2,3,4,7,9,12,13,14])
    AdGraph[9] = array([3,4,8,13,14])

    AdGraph[10] = array([5,6,11,15,16])
    AdGraph[11] = array([5,6,7,10,12,15,16,17])
    AdGraph[12] = array([6,7,8,11,13,16,17,18])
    AdGraph[13] = array([7,8,9,12,14,17,18,19])
    AdGraph[14] = array([8,9,13,18,19])

    AdGraph[15] = array([10,11,16,21])
    AdGraph[16] = array([10,11,12,15,17,21,22])
    AdGraph[17] = array([11,12,13,16,18,21,22,23])
    AdGraph[18] = array([12,13,14,17,19,22,23,24])
    AdGraph[19] = array([13,14,18,23,24])

    AdGraph[21] = array([15,16,17,22])
    AdGraph[22] = array([16,17,18,21,23])
    AdGraph[23] = array([17,18,19,22,24])
    AdGraph[24] = array([18,19,23])
    
    #-------------------------------------------------------------
    #Plotting stuff
    fig = plt.figure(figsize=(12,16.5))
    ax1 = fig.add_subplot(321)    #Latencies between PW AW and RW
    ax2 = fig.add_subplot(323)    #Latencies in PW ordered by respones
    ax3 = fig.add_subplot(324)    #Latencies in PW ordered from min to max
    ax4 = fig.add_subplot(325)    #Latencies in PW ordered from min to max

        
    # Hide the right and top spines
    ax1.spines['right'].set_visible(False)
    ax1.spines['top'].set_visible(False)
    # Only show ticks on the left and bottom spines
    ax1.yaxis.set_ticks_position('left')
    ax1.xaxis.set_ticks_position('bottom')
    
    # Hide the right and top spines
    ax2.spines['right'].set_visible(False)
    ax2.spines['top'].set_visible(False)
    # Only show ticks on the left and bottom spines
    ax2.yaxis.set_ticks_position('left')
    ax2.xaxis.set_ticks_position('bottom')
         
    # Hide the right and top spines
    ax3.spines['right'].set_visible(False)
    ax3.spines['top'].set_visible(False)
    # Only show ticks on the left and bottom spines
    ax3.yaxis.set_ticks_position('left')
    ax3.xaxis.set_ticks_position('bottom')
    
    # Hide the right and top spines
    ax4.spines['right'].set_visible(False)
    ax4.spines['top'].set_visible(False)
    # Only show ticks on the left and bottom spines
    ax4.yaxis.set_ticks_position('left')
    ax4.xaxis.set_ticks_position('bottom')
        
    colors = ['blue','green','red','yellow','orange','cyan','magenta','greenyellow','black','gray', 
              'purple','salmon','brown', 'beige','chocolate','dimgray','ivory','seagreen','lavender','palegreen']
    
    lt=['-','--']
    lts=['-o','-s']
    #-------------------------------------------------------------
    #-------------------------------------------------------------
      
    data_pw = []
    data_aw = []
    datos_PWm = []
    datos_PWM = []
    datos_PWmax = []
    Pref = zeros(8)
    neucol=-1      #initialize colors  
    
    for neuron in sorted(PW.keys())[:]:        #loop ordered units
        neucol+=1
        if len(np.where(STC_on[neuron]==True)[0])>0:  #if list not empty
            #compute latencies
            PW0=Latencies[neuron][0][PW[neuron]]
            PW1=Latencies[neuron][1][PW[neuron]]
            PWmin = min(PW0,PW1)
            PWmax = max(PW0,PW1)
            #------------------------------------------------------
            #plotting options. If positive wins, then it's square, if not circle
            if PW0<PW1:
                dotts = [0,1]
            else:
                dotts = [1,0]
            #------------------------------------------------------
            #Plot PW dominant and PW oposite
            if isnan(PWmin):     #in case one direction does not respond
                PWmin = np.nanmin([PW0,PW1])
                PWmax = float('nan')
                datos_PWm =np.append(datos_PWm,PWmin)
                ax2.plot([0],PWmin, lts[dotts[1]] ,color= colors[neucol])
            else:
                if ResRC[neuron][PW[neuron]]==1:          #ask for positive dominant PW
                    datos_PWm =np.append(datos_PWm,PW0)
                    datos_PWM =np.append(datos_PWM,PW1)
                    ax2.plot([0,1],[PW0,PW1],lt[0],color= colors[neucol],ms=4)
                    ax2.plot([0],PW0, lts[dotts[1]] ,color= colors[neucol])
                    ax2.plot([1],PW1, lts[dotts[0]] ,color= colors[neucol])
                else:
                    datos_PWm =np.append(datos_PWm,PW1)
                    datos_PWM =np.append(datos_PWM,PW0)
                    ax2.plot([0,1],[PW1,PW0],lt[0],color= colors[neucol],ms=4)
                    ax2.plot([0],PW1, lts[dotts[1]] ,color= colors[neucol])
                    ax2.plot([1],PW0, lts[dotts[0]] ,color= colors[neucol])
            #------------------------------------------------------
            #save data for boxplots   
            data_pw = np.append(data_pw,PWmin)
            datos_PWmax = np.append(datos_PWmax,PWmax)
            
            if ResRC[neuron][PW[neuron]]==1: Pref[0] +=1 
            else: Pref[1] +=1 
            
            #------------------------------------------------------
            #here I plot the PW in first and third graphs
            if PW[neuron]!=20:
                #if pos direction is PW dominant, I plot it with a square and I label it 
                #if neg direction is PW dominant, I plot it with a circle and I label it 
                ax1.plot([0],PWmin,lts[dotts[1]],color= colors[neucol],label ='Nrn_'+str(neuron))
                                
                ax3.plot([0,1],[PWmin,PWmax],lt[0],color= colors[neucol],ms=4)
                ax3.plot([0],PWmin, lts[dotts[1]] ,color= colors[neucol])
                ax3.plot([1],PWmax, lts[dotts[0]] ,color= colors[neucol])
            
            #------------------------------------------------------
            #Loop and plot adjacent whiskers that are active
            for aw in AdGraph[PW[neuron]]:
                if STC_on[neuron][aw]==True:
                
                    aw0=Latencies[neuron][0][aw]
                    aw1=Latencies[neuron][1][aw]
                    data_aw = np.append(data_aw,[aw0,aw1])
                    
                    if ResRC[neuron][aw]==1: 
                        Pref[2] +=1 
                        if ResRC[neuron][PW[neuron]]==1:
                            Pref[6]+=1
                        else:
                            Pref[7]+=1
                    else: 
                        Pref[3] +=1 
                        if ResRC[neuron][PW[neuron]]==-1:
                            Pref[6]+=1
                        else:
                            Pref[7]+=1
                 
                    #plot the first aw with their corresponding marks 
                    ax1.plot([1],aw0,lts[1*(ResponseRC[neuron][aw]==1)],color= colors[neucol])
                    ax1.plot([1],aw1,lts[1*(ResponseRC[neuron][aw]==-1)],color= colors[neucol])
                    #then I join points
                    ax1.plot([0,1],[PWmin,aw0],lt[dotts[0]],color= colors[neucol],ms=4)
                    ax1.plot([0,1],[PWmin,aw1],lt[dotts[1]],color= colors[neucol],ms=4)
            #------------------------------------------------------
            #Loop and plot r whiskers that are active
            for rw in arange(25):
                if rw not in AdGraph[PW[neuron]] and rw!=PW[neuron] and STC_on[neuron][rw]==True:
                    rw0 = Latencies[neuron][0][rw] 
                    rw1 = Latencies[neuron][1][rw] 
                    
                    ax1.plot([2],rw0,lts[1*(ResponseRC[neuron][rw]==1)],color= colors[neucol])
                    ax1.plot([2],rw1,lts[1*(ResponseRC[neuron][rw]==-1)],color= colors[neucol])
                    ax1.plot([0,2],[PWmin,rw0],lt[dotts[0]],color= colors[neucol],ms=4)
                    ax1.plot([0,2],[PWmin,rw1],lt[dotts[1]],color= colors[neucol],ms=4)

    
    #---------------------------
    #titles
    if grupete ==1:
        fig.suptitle( titles + '_Latencies_multiunits',fontsize=16)
    else:
        fig.suptitle( titles + '_Latencies' ,fontsize=16)
             
        fig.add_subplot(ax1)
        fig.add_subplot(ax2)
    #---------------------------
    #boxplots
    ax1.boxplot([data_pw,data_aw],positions = [-0.25,1.25])
    ax2.boxplot(datos_PWm,positions = [-0.25])
    ax2.boxplot(datos_PWM,positions = [1.25])
    ax3.boxplot(data_pw,positions = [-0.25])
    ax3.boxplot(datos_PWmax,positions = [1.25])
    
    
    Pref[4] = Pref[0]+Pref[2]
    Pref[5] = Pref[1]+Pref[3]
    #---------------------------
    #ylims and labels
    y_lims = ax1.get_ylim()
    ax1.set_ylim([6,36])
    #ax1.set_ylim(y_lims)
    y_lims = ax2.get_ylim()
    ax2.set_ylim([8,35])
    #ax2.set_ylim(y_lims)
    ax3.set_ylim([8,35])
    
    ax1.set_ylabel('Latencies (ms)')
    ax2.set_ylabel('Latencies (ms)')
    ax3.set_ylabel('Latencies (ms)')
    ax4.set_ylabel('Number of whiskers')

    #---------------------------
    #xlims and labels
    ax1.set_xlim([-0.5, 2.2])
    ax2.set_xlim([-0.5, 2.2])
    ax3.set_xlim([-0.5, 2.2])
    ax4.set_xlim([-0.5, 2.2])

    ind = arange(3)
    ax1.set_xticks(ind)   
    ax1.set_xticklabels( ['PW', 'AW', 'RW'] )
    
    ind = arange(2)
    ax2.set_xticks(ind)   
    ax2.set_xticklabels( ['PWmaxResp', 'PWsecond'] )
    
    ax3.set_xticks(ind)   
    ax3.set_xticklabels( ['PWmin', 'PWmax'] )
    
    ind = arange(-0.4,2.1,0.35)
    ax4.set_xticks(ind)   
    ax4.set_xticklabels( ['','PWup-down','','AWup-down','','TOTup-down','','PW =/op AW'] )
    for i in arange(4): ind[2*i+1] = ind[2*i+1]-0.1
    ax4.bar(ind,Pref,width=0.2)
    #---------------------------
    #legend and texts
    ax1.legend(bbox_to_anchor=(1.45, 1.2), loc=1, borderaxespad=0.)
    
    ax1.text(-0.3, 37,"mean    = %.1f"% nanmean(data_pw))
    ax1.text(-0.3, 36,"median = %.1f"% nanmedian(data_pw))
    ax1.text(-0.3, 35,"stdev    =   %.1f"% nanstd(data_pw))
    ax1.text(0.9, 37,"mean    = %.1f"% nanmean(data_aw))
    ax1.text(0.9, 36,"median = %.1f"% nanmedian(data_aw))
    ax1.text(0.9, 35,"stdev    =   %.1f"% nanstd(data_aw))
    
    ax2.text(-0.3, 36,"mean    = %.1f"% nanmean(datos_PWm))
    ax2.text(-0.3, 35,"median = %.1f"% nanmedian(datos_PWm))
    ax2.text(-0.3, 34,"stdev    =   %.1f"% nanstd(datos_PWm))
    ax2.text(0.9, 36,"mean    = %.1f"% nanmean(datos_PWM))
    ax2.text(0.9, 35,"median = %.1f"% nanmedian(datos_PWM))
    ax2.text(0.9, 34,"stdev    =   %.1f"% nanstd(datos_PWM))
    
    ax3.text(-0.3, 36,"mean    = %.1f"% nanmean(data_pw))
    ax3.text(-0.3, 35,"median = %.1f"% nanmedian(data_pw))
    ax3.text(-0.3, 34,"stdev    =   %.1f"% std(data_pw))
    ax3.text(0.9, 36,"mean    = %.1f"% nanmean(datos_PWmax))
    ax3.text(0.9, 35,"median = %.1f"% nanmedian(datos_PWmax))
    ax3.text(0.9, 34,"stdev    =   %.1f"% nanstd(datos_PWmax))
    
    ax4.legend(bbox_to_anchor=(1.45, 1.), loc=1, borderaxespad=0.)
    #---------------------------
    #save figure
    if grupete ==1:                  
        fig.savefig(pdf_files_directory + titles + '_Latencies_multi.pdf', format='pdf')    
    else:
        fig.savefig(pdf_files_directory + titles + '_Latencies.pdf', format='pdf')    


#----------------------------------------------------------------------------------------
#----------------------------------------------------------------------------------------          
#----------------------------------------------------------------------------------------
# DISPLAY STA
#----------------------------------------------------------------------------------------
# Plot a single STA
def display_STA(STAneuron, origin,fig,inner_grid,neuron,numspikes,STC_on,PW) :
    
    height =np.max(abs(STAneuron))
    ymax = 1.02 * height
    
    for i in range(25) :
        if i == 0 :
            ax1 = Subplot(fig, inner_grid[i])     
            ax1.set_xticks([])
            ax1.set_yticks([])
        
        elif i==20:
            ax1 = Subplot(fig,inner_grid[i],sharex=ax1,sharey=ax1)
            ax1.spines['right'].set_linewidth(0.3)
            ax1.spines['top'].set_linewidth(0.3)
            ax1.spines['left'].set_linewidth(0.3)
            ax1.spines['bottom'].set_linewidth(0.3)
        else :
            ax1 = Subplot(fig,inner_grid[i],sharex=ax1,sharey=ax1)
            xticks([],[])  #gets rid of the x ticks and numbers
            yticks([],[])  #gets rid of the y ticks and numbers
       
    
        if PW==i:
            ax1.set_axis_bgcolor('#dddddd')    
        elif i!=20:
            ax1.spines['right'].set_visible(False)
            ax1.spines['top'].set_visible(False)
            ax1.spines['left'].set_visible(False)
            ax1.spines['bottom'].set_visible(False)
        if STC_on[i]== True:
            ax1.set_axis_bgcolor('#dddddd')    
    
        ax1.set_xlim([0, int(STAneuron.shape[1])]) 
        ax1.axvline(origin, color = 'r', linewidth=0.5)
        ax1.axhline(0, color = 'r', linewidth=0.5)
        ax1.plot(STAneuron[i,:])
        
        ax1.set_ylim([-ymax, ymax])
        
        if i==4: ax1.set_title('ymax =' + str( np.around(ymax,decimals = 3) ),fontsize=8)
        if i ==1: ax1.set_title('STA_Nrn' + str(neuron) + '_Stim' + choice+ '_'+ str(int(numspikes)),fontsize=9)

        fig.add_subplot(ax1)

#----------------------------------------------------------------------------------------
# SAVE ALL STA OF RECORDING
#----------------------------------------------------------------------------------------
# Plot and save all STAs pdfs
def save_all_STAs_of_recording(STA_Table, spike_counts, choice, pdf_files_directory, origin,STC_on,PW,titles,grupete) :
    
    fig = plt.figure(figsize=(12,16.5))
    nrns = len(STA_Table.keys())
    if nrns <16: 
        layout = [5,3]
    else: layout = [nrns//3+(nrns%3!=0),3]
    outer_grid = gridspec.GridSpec(layout[0], layout[1], wspace=0.1, hspace=0.2)
    
    ii=0
    
    orderneurons = np.sort(list(STA_Table.keys()))
    
    for neuron in  orderneurons:
      
        inner_grid = gridspec.GridSpecFromSubplotSpec(5,5,subplot_spec=outer_grid[ii], wspace=0.1, hspace=0.1)
        
        numspikes=spike_counts[neuron]
        display_STA(STA_Table[neuron], origin,fig,inner_grid,neuron,numspikes,STC_on[neuron],PW[neuron])
      
        if grupete ==1:
            fig.suptitle(titles + '_STA_Stim' + choice +'_multiunits',fontsize=16)
        else:
            fig.suptitle(titles + '_STA_Stim' + choice ,fontsize=16)
        ii+=1
        
    if grupete ==1:                  
        fig.savefig(pdf_files_directory + titles + '_STA_Stim' + choice + '_multi.pdf', format='pdf')
    else:
        fig.savefig(pdf_files_directory + titles + '_STA_Stim' + choice + '.pdf', format='pdf')

        
#----------------------------------------------------------------------------------------
#----------------------------------------------------------------------------------------
#----------------------------------------------------------------------------------------
# SAVE ALL F C U
#----------------------------------------------------------------------------------------
def save_all_FCU_of_recording(spike_counts,pdf_files_directory,titles,grupete):
    
    fig3 = plt.figure(figsize=(12,16.5))
    nrns = len(list(spike_counts['F'].keys()))
    if nrns <16: 
        layout = [5,3]
    else: layout = [nrns//3+(nrns%3!=0),3]
    outer_grid = gridspec.GridSpec(layout[0], layout[1], wspace=0.3, hspace=0.2)
    
    orderneurons = np.sort(list(spike_counts['F'].keys()))
    
    global UFC
    
    ind=np.arange(3)
    width= 0.5
       
    fig1 = plt.figure(figsize=(12,16.5),frameon=False)
           
    ax1 = fig1.add_subplot(321)    
    ax2 = fig1.add_subplot(323)
    
    ax1.set_xlim([-0.2, 2.2])
    ax1.set_xticks(ind)   
    ax1.set_xticklabels( ['U', 'F', 'C'] )
    
    ax2.set_xlim([-0.2, 2.2])
    ax2.set_xticks(ind)   
    ax2.set_xticklabels( ['U', 'F', 'C'] )
    
    
    ax1.set_ylim([0, 5000])        
    ax2.set_ylim([0, 1.5])     
    
    # Hide the right and top spines
    ax1.spines['right'].set_visible(False)
    ax1.spines['top'].set_visible(False)
    # Only show ticks on the left and bottom spines
    ax1.yaxis.set_ticks_position('left')
    ax1.xaxis.set_ticks_position('bottom')
    
    # Hide the right and top spines
    ax2.spines['right'].set_visible(False)
    ax2.spines['top'].set_visible(False)
    # Only show ticks on the left and bottom spines
    ax2.yaxis.set_ticks_position('left')
    ax2.xaxis.set_ticks_position('bottom')
    
    ii=0
    for neuron in orderneurons :     
        
        inner_grid = gridspec.GridSpecFromSubplotSpec(1,1,subplot_spec=outer_grid[ii], wspace=0.1, hspace=0.1)

        i=0
        ax3 = Subplot(fig3, inner_grid[i])     
        ax3.set_xticks([])
        #ax3.set_yticks([])
                
        numspikes= [spike_counts['U'][neuron]/UFC[0]*360, spike_counts['F'][neuron]/UFC[1]*360,spike_counts['C'][neuron]/UFC[2]*360]
                        
        ax3.set_xlim([-0.25, 2.8])
        ax3.set_xticks(ind+width/2)   
        ax3.set_xticklabels( ['U', 'F', 'C'] )
        
        ax3.bar(ind,numspikes,width)
        
        # Hide the right and top spines
        ax3.spines['right'].set_visible(False)
        ax3.spines['top'].set_visible(False)
        # Only show ticks on the left and bottom spines
        ax3.yaxis.set_ticks_position('left')
        ax3.xaxis.set_ticks_position('bottom')
        
        if (numspikes[0]>numspikes[1] and numspikes[0]>numspikes[2]):
            lt = '--s'
        elif numspikes[1]>numspikes[2]:
            lt = '-o'
        else:
            lt = ':d'
         
        if (x>300 and x< 10000 for x in numspikes):
            ax1.plot([0,1,2],numspikes,lt,label ='Nrn_'+str(neuron))
            ax2.plot([0,1,2],[x/numspikes[1] for x in numspikes ],lt,label ='Nrn_'+str(neuron))
        
        if grupete ==1:
            fig3.suptitle( titles + '_UFC_multiunits',fontsize=16)
        else:
            fig3.suptitle( titles + '_UFC' ,fontsize=16)
        ii+=1
        
        ax3.set_title('Nrn_' + str(neuron) + '_Tot' +str(int(sum(numspikes))),fontsize=9)
        
        fig3.add_subplot(ax3)

    if grupete ==1:
            fig1.suptitle(titles + '_UFC_multiunits',fontsize=16)
    else:
            fig1.suptitle(titles + '_UFC' ,fontsize=16)
    ax1.set_title('UFC relation',fontsize=12)
    ax2.set_title('UFC relation normalized',fontsize=12)
    
    ax1.set_ylabel('Spike counts')
    ax1.legend(bbox_to_anchor=(1.55, 1), loc=1, borderaxespad=0.)
    
    #fig1.add_subplot(ax1)    
    #fig1.add_subplot(ax2)
          
    if grupete ==1:                  
        fig1.savefig(pdf_files_directory + titles + 'UFC_all_multi.pdf', format='pdf')    
        fig3.savefig(pdf_files_directory + titles + '_UFC_multi.pdf', format='pdf')
    else:
        fig1.savefig(pdf_files_directory + titles + 'UFC_all.pdf', format='pdf')    
        fig3.savefig(pdf_files_directory + titles + '_UFC.pdf', format='pdf')        

#----------------------------------------------------------------------------------------
#----------------------------------------------------------------------------------------
#----------------------------------------------------------------------------------------
# DISPLAY STC
#----------------------------------------------------------------------------------------
#Plot eigenvalues
def display_STC_eigs(STC_eigs, Spike_counts, whiskers, origin, choice, pdf_files_directory,titles,grupete) :
    close("all")
    
    fig = plt.figure(figsize=(12,16.5))
    nrns = len(STC_eigs.keys())
    if nrns <16: 
        layout = [5,3]
    else: layout = [nrns//3+(nrns%3!=0),3]
    outer_grid = gridspec.GridSpec(layout[0], layout[1], wspace=0.2, hspace=0.4)
 
    orderneurons = np.sort(list(STC_eigs.keys()))
    
    ii=0
    clf()
    for neuron in orderneurons:
        noprint=0
    
        inner_grid = gridspec.GridSpecFromSubplotSpec(1,1,subplot_spec=outer_grid[ii], wspace=0.1, hspace=0.1)
        ax1 = Subplot(fig, inner_grid[0])     
          
        whiskerinds = np.where(whiskers[neuron]==True)[0]
        numwhiskers = len(whiskerinds)
        if np.where(whiskers[neuron]==True)[0].size==0 :
            #whiskerinds = np.arange(25, dtype='int')
            whiskinds = np.array([20])                   #uncomment to speed up and not evaluate neurons without significant whiskers
            numwhiskers = 1                               #idem
            noprint = 1

        numspikes = Spike_counts[neuron]
        
        eigvals = STC_eigs[neuron].eigvals
        
        inds = np.argsort(eigvals)[::-1]
        ploteigvals = eigvals[inds]

        eigvals = eigvals[inds]
        
        #figure()
        ax1.plot(ploteigvals, 'bo',markersize=5,)
        fig.add_subplot(ax1)
        
        ax1.set_xlim(left = 0)
        ax1.set_title('Nrn_' + str(neuron) ,fontsize=9)
        ax1.set_xlabel('#Eigenvalue')
        ax1.tick_params(axis='both', which='major', labelsize=7)
        ax1.yaxis.set_major_formatter(mtick.FormatStrFormatter('%.0e'))
        
        if grupete ==1:
            fig.suptitle(titles + '_STC_Eigs_Stim' + choice +'_multiunits',fontsize=16)
        else:
            fig.suptitle(titles + '_STC_Eigs_Stim' + choice ,fontsize=16)
        
        close("all")
        ii+=1   
        if noprint==0:    
            if grupete ==1:                  
                fig.savefig(pdf_files_directory + titles + '_STC_Eigs_Stim' + choice + '_multi.pdf', format='pdf')
            else:
                fig.savefig(pdf_files_directory + titles + '_STC_Eigs_Stim' + choice + '.pdf', format='pdf')

#----------------------------------------------------------------------------------------
# DISPLAY STC
#----------------------------------------------------------------------------------------        
# Plot all STC and save them
def display_STC(STC_eigs, Spike_counts, whiskers, numfilters, origin, choices, pdf_files_directory,PW,titles,grupete) :
    
    
    nrns = len(STC_eigs.keys())
    if numfilters<5:
        layout = [5,3]
    else:
        layout = [numfilters,3]

    outer_grid = gridspec.GridSpec(layout[0], layout[1], wspace=0.2, hspace=0.2)
 
    orderneurons = np.sort(list(STC_eigs['F'].keys()))    
    
    for neuron in orderneurons:
        clf()
        fig = plt.figure(figsize=(12,16.5))
    
        whiskerinds = np.where(whiskers[neuron]==True)[0]
        numwhiskers = len(whiskerinds)
        if np.where(whiskers[neuron]==True)[0].size==0 :
            #whiskerinds = np.arange(25, dtype='int')
            whiskinds = np.array([20])                   #uncomment to speed up and not evaluate neurons without significant whiskers
            numwhiskers = 1                               #idem
            continue
        
        j=0
        for choice in choices:
            j+=1                     #column U C or F
        
            numspikes = Spike_counts[choice][neuron]
        
            eigvecs = STC_eigs[choice][neuron].eigvecs
            eigvals = STC_eigs[choice][neuron].eigvals

            inds = np.argsort(eigvals)[::-1]
            indsplot = np.argsort(eigvals)[::-1]
            ploteigvals = eigvals[inds]

            inds = inds[0:numfilters]
            eigvals = eigvals[inds]
            eigvecs = eigvecs[:, inds]
            eigvecs = np.reshape(eigvecs, [eigvecs.shape[0]/numwhiskers, numwhiskers, numfilters], 'F')

            for fil in np.arange(numfilters, dtype='int') :
                iii=(j-1)+fil*3              #subplot number
               
                inner_grid = gridspec.GridSpecFromSubplotSpec(5,5,subplot_spec=outer_grid[iii], wspace=0.1, hspace=0.1)
                for i in np.arange(25, dtype='int') :

                    if i == 0:
                        ax2 = Subplot(fig, inner_grid[i])     
                        ax2.set_xticks([])
                        ax2.set_yticks([])      
                        ax2.spines['right'].set_visible(False)
                        ax2.spines['top'].set_visible(False)
                        ax2.spines['left'].set_visible(False)
                        ax2.spines['bottom'].set_visible(False)
                    elif i == PW[neuron]:
                        ax2 = Subplot(fig, inner_grid[i],sharex=ax2,sharey=ax2)     
                        ax2.set_xticks([])
                        ax2.set_yticks([])        
                    else :
                        ax2 = Subplot(fig,inner_grid[i],sharex=ax2,sharey=ax2)
                        ax2.spines['right'].set_visible(False)
                        ax2.spines['top'].set_visible(False)
                        ax2.spines['left'].set_visible(False)
                        ax2.spines['bottom'].set_visible(False)

                        ax2.set_xticks([])
                        ax2.set_yticks([])        

                    if any(i==whiskerinds) :
                        tempind = np.where(whiskerinds==i)[0]
                        ax2.plot(eigvecs[:,tempind,fil])
                        ymax = 1.02 * np.max(abs(eigvecs))
                        ylim(-ymax, ymax)

                    ax2.axvline(origin, color = 'r', linewidth=0.5)
                    ax2.axhline(0, color = 'r', linewidth=0.5)

                    if i==2 and iii<3:
                        ax2.set_title('Eigval' + str(fil) + '_' + 'stim_'+ choice + '_'+ str(numspikes),fontsize=12)
                    elif i==2:
                        ax2.set_title('Eigval' + str(fil),fontsize=12)
                    fig.add_subplot(ax2)
        if grupete ==1:
            fig.suptitle(titles + '_STC_Nrn' + str(neuron) +'_multiunits',fontsize=16)
        else:
            fig.suptitle(titles + '_STC_Nrn' + str(neuron) ,fontsize=16)
        fig.savefig(pdf_files_directory + 'STC/' +  titles + '_STC_Nrn' + str(neuron) +  '.pdf', format='pdf')
        close("all")

# Experiment Files and Folders

In [1]:
#Experiment numbers
ExpeNum = [18,20,21,22,23]

#Folders for measurements and experiments
m12 = ['m1s1','m1s2','m1s3','m1s4','m2s1','m2s2','m2s3','m2s4']
m13 = ['m1s1','m1s2','m1s3','m1s4','m3s1','m3s2','m3s3','m3s4']

m164 = ['m1s1','m1s2','m1s3','m1s4','m1s5','m1s6','m1s7','m1s8']
m264 = ['m2s1','m2s2','m2s3','m2s4','m2s5','m2s6','m2s7','m2s8']


Expe={}
Vtags={}
Stim={}
for num in ExpeNum: 
    Expe[num] = dict()
    Vtags[num] = dict()
    
#Kwik files    
files18 = ['MEAS-150630-1_ele01_ele08.kwik',
                    'MEAS-150630-1_ele09_ele16.kwik',
                    'MEAS-150630-1_ele17_ele24.kwik',
                    'MEAS-150630-1_ele25_ele32.kwik',
                    'MEAS-150630-2_ele01_ele08.kwik',
                    'MEAS-150630-2_ele09_ele16.kwik',
                    'MEAS-150630-2_ele17_ele24.kwik',
                    'MEAS-150630-2_ele25_ele32.kwik']

files20 = ['MEAS-150707-1_ele01_ele08.kwik',
                    'MEAS-150707-1_ele09_ele16.kwik',
                    'MEAS-150707-1_ele17_ele24.kwik',
                    'MEAS-150707-1_ele25_ele32.kwik',
                    'MEAS-150707-23_ele01_ele08.kwik',
                    'MEAS-150707-23_ele16_ele09.kwik',
                    'MEAS-150707-23_ele17_ele24.kwik',
                    'MEAS-150707-23_ele25_ele32.kwik']
    
files21 = ['MEAS-150709-1-ordered_ele01_ele08.kwik',
                    'MEAS-150709-1-ordered_ele09_ele16.kwik',
                    'MEAS-150709-1-ordered_ele17_ele24.kwik',
                    'MEAS-150709-1-ordered6_ele25_ele30.kwik',
                    'MEAS-150709-23-ordered_ele01_ele08.kwik',
                    'MEAS-150709-23-ordered_ele09_ele16.kwik',
                    'MEAS-150709-23-ordered_ele17_ele24.kwik',
                    'MEAS-150709-23-ordered6_ele25_ele30.kwik']
                   
            
files22 = ['MEAS-150716-12_ele01_ele08.kwik',
                    'MEAS-150716-12_ele09_ele16.kwik',
                    'MEAS-150716-12_ele17_ele24.kwik',
                    'MEAS-150716-12_ele25_ele32.kwik',
                    'MEAS-150716-3_ele01_ele08.kwik',
                    'MEAS-150716-3_ele09_ele16.kwik',
                    'MEAS-150716-3_ele17_ele24.kwik',
                    'MEAS-150716-3_ele25_ele32.kwik']

files23 = ['MEAS-151027-1_ele01_ele08.kwik',
                    'MEAS-151027-1_ele09_ele16.kwik',
                    'MEAS-151027-1_ele17_ele24.kwik',
                    'MEAS-151027-1_ele25_ele32.kwik',
                    'MEAS-151027-1_ele33_ele40.kwik',
                    'MEAS-151027-1_ele41_ele48.kwik',
                    'MEAS-151027-1_ele49_ele56.kwik',
                    'MEAS-151027-1_ele57_ele64.kwik',
                    'MEAS-151027-2_ele01_ele08.kwik',
                    'MEAS-151027-2_ele09_ele16.kwik',
                    'MEAS-151027-2_ele17_ele24.kwik',
                    'MEAS-151027-2_ele25_ele32.kwik',
                    'MEAS-151027-2_ele33_ele40.kwik',
                    'MEAS-151027-2_ele41_ele48.kwik',
                    'MEAS-151027-2_ele49_ele56.kwik',
                    'MEAS-151027-2_ele57_ele64.kwik']

#Vtag files 
Vtags[18] = ['MEAS-150630-1_Vtag1.dat','MEAS-150630-2_Vtag1.dat']
Vtags[20] = ['MEAS-150707-1_Vtag1.dat','MEAS-150707-23_Vtag1.dat']
Vtags[21] = ['MEAS-150709-1_Vtag1.dat','MEAS-150709-23_Vtag1.dat']
Vtags[22] = ['MEAS-150716-12_Vtag1.dat','MEAS-150716-3_Vtag1.dat']
Vtags[23] = ['MEAS-151027-1_Vtag1.dat','MEAS-151027-2_Vtag1.dat']

#Build Experiment dictionary
i=0
for meas in m12:
    Expe[18][meas] = files18[i]
    Expe[21][meas] = files21[i]
    i+=1
i=0
for meas in m13:
    Expe[20][meas] = files20[i]
    Expe[22][meas] = files22[i]
    i+=1

i=0    
for meas in np.append(m164,m264):
    Expe[23][meas] = files23[i]
    i+=1
    
    
#Stimulus type
for expe in ExpeNum:
    Stim[expe] = 'big_STIM'

Stim[23] = 'big_STIM_FC_corrected'
    
#--------------------------------------------------------------------------------
#Folders
rootF = '/home/matias/WORKSPACE/'    
stimFolder = rootF +'STIM/'

NameError: name 'np' is not defined

# Define and load data files from experiments

In [28]:
global binname, textname, UFC
#---------------------------------------------------------------------------------------
SelExp = [23]   #Expe                                        #select experiment numbers!
grupete = [2]                                                #select cluster groups! 2 for good clusters 1 for multiunits
Measurements = [m164[0]]#m12[-4:]#['m1s1','m1s2','m1s3','m1s4']        #select measurement and/or shanks!
loadVtag = 1                                                 #number of shank in which to load Vtag
choices = ['F','C','U']                                      #select stimulus type
dirs =[]
ploteo = [1,0,0,0,0,0]                                         #1 to make plots:  psth,latencies, sta, ufc, stc, stcnrns
computePSTH = 1                                              #1 to do PSTH
computeLat = 0                                              #1 to do PSTH
computeSTA= 0                                                #1 to do STA
computeSTC = 0                                               #1 to do STC

#--------------------------------------------------------------------------------
#loop experiments
for expe in SelExp:
    
    #Measurements = sorted(Expe[expe])                         #uncommento to select all
    
    print(expe)

    binname= stimFolder + Stim[expe] + '/Stimulus_UCC.bin'
    textname=stimFolder + Stim[expe] + '/Stimulus_UCC.txt'
    
    #--------------------------------------------------------------------------------
    #loop goodunits and multiunits
    for group in grupete:   #2 for good clusters 1 for multiunits
        
        #folder names
        if group ==2:
            dirs  = [rootF + 'OUTPUT/PDFall',rootF + 'OUTPUT/PDFall/STC']
        if group ==1:
            dirs  = [rootF + 'OUTPUT/PDFallM',rootF + 'OUTPUT/PDFallM/STC']
        if group ==3:
            dirs  = [rootF + 'OUTPUT/PDFallU',rootF + 'OUTPUT/PDFallU/STC']
     
        #--------------------------------------------------------------------------------
        #loop measurements and shanks
        measurements = Expe[expe]                            
                
        for meas in Measurements:           
            
            print(meas)
    
            #select datafile
            sp_file = rootF + 'EXP_' + str(expe) +'/'+ meas +'/'+ measurements[meas]
            #select Vtag
            if int(meas[1])<2: measV = 0    #select Vtag
            else: measV=1
            bin_file = rootF + 'EXP_' + str(expe) +'/' + Vtags[expe][measV]
            #--------------------------------------------------------------------------------
            #load data
            Spikes, sampling_freq = readkwikinfo(sp_file, group)
            #load Vtag and stimulus
            if int(meas[1])==1 and int(meas[3])==loadVtag:               #load the stimulus once every 4 shanks
                stim = []
                stimtype=[]
                stim,stimtype = read_stimulus(expe,1)
                
                Vtag1 =[]
                Vtag1 = np.fromfile(file=bin_file, dtype=np.int16)
                start_and_stops = Vtag1[1:] - Vtag1[:-1]
                starts = (where(start_and_stops==1)[0]-2999)/float(sampling_freq) # time in seconds
                stops = (where(start_and_stops==-1)[0]+4110)/float(sampling_freq) # time in seconds
            
            if (int(meas[1])==2 or int(meas[1])==3) and int(meas[3])==loadVtag:
                stim = []
                stimtype=[]
                stim,stimtype = read_stimulus(expe,2)

                Vtag1 =[]
                Vtag1 = np.fromfile(file=bin_file, dtype=np.int16)
                start_and_stops = Vtag1[1:] - Vtag1[:-1]
                starts = (where(start_and_stops==1)[0]-2999)/float(sampling_freq) # time in seconds
                stops = (where(start_and_stops==-1)[0]+4110)/float(sampling_freq) # time in seconds
                        
            if len(Spikes.keys())>0:                              #do only if there are clusters
                #--------------------------------------------------------------------------------
                #create output folders
                
                for dir in dirs:
                    if not os.path.exists(dir):
                        os.makedirs(dir) 

                dire = dirs[0] +'/'
                titles = 'Exp'+ str(expe) + '_Meas_' + meas[1] + '_Shank_' + meas[3]
                #--------------------------------------------------------------------------------
                #Build PSTHs
                
                t_before = .005  
                t_after = .045
                thresh = 0.37     #Act mod activity between 10,45ms and -10,10 ms (from -1 to 1)
                activity = 40    #minimun amount of activity to get an active whisker from 10 to 45ms
                chancelevel = 1/25*1.4   #how much activity above chance in response area (10-45ms)
                
                if computePSTH ==1:
                    PSTH_spikes_counts, hist_output = BuildPSTH(Vtag1,stim,stimtype, Spikes, sampling_freq, t_before, t_after,starts,stops)
                STC_on, PW ,ActMod,ActMod2= GetWhiskers(hist_output, t_before,t_after,thresh,activity,chancelevel)
                #--------------------------------------------------------------------------------
                #Plot PSTH
                if ploteo[0]==1: display_all_PSTHs_of_recording(hist_output, PSTH_spikes_counts, dire, t_before, t_after,group,STC_on,PW,titles)
                #-----------------------------
                #Get Latencies
                
                zscore = 3         #stdev of first two consecutive bins that determine the latency
                zscore_th = 1.8
                bin_window = 5
                
                if computeLat:    
                    Latencies, LatMean, ResponseRC = BuildLatencies(hist_output,STC_on,t_before,t_after,zscore_th,bin_window)
                #--------------------------------------------------------------------------------
                #Plot Latencies
                    if ploteo[1]==1: display_Latencies(PW,STC_on,ResponseRC,Latencies,titles,grupete,dire)
                #--------------------------------------------------------------------------------
                #Build STAs
                
                if computeSTA==1:
                    STA_Tables = {}
                    Counts_Tables = {}
                    Spike_Tables = {}

                    t_before = .045
                    t_after = .005

                    #Loop all stimulus types
                    for choice in choices:
                        Counts_Tables[choice], Spike_Tables[choice] = BuildSpikeTables(Spikes, Vtag1,stim,stimtype, sampling_freq, t_before, t_after, choice,starts,stops)
                        STA_Tables[choice] = Build_STA(Spike_Tables[choice])
                #--------------------------------------------------------------------------------
                #Plot STAs
                if ploteo[2]==1:
                    for choice in choices:
                        save_all_STAs_of_recording(STA_Tables[choice], Counts_Tables[choice], choice, dire, t_before*1000,STC_on,PW,titles,group)
                #--------------------------------------------------------------------------------
                #Plot UFC
                if ploteo[3]==1:save_all_FCU_of_recording(Counts_Tables,dire,titles,group)
                #--------------------------------------------------------------------------------
                #Build STCs
                
                if computeSTC==1:
                    numsamples = 10000
                    cutoff = 0.25

                    STC_eigs = {}
                    STC_covmats = {}
                    W = {}
                    Nstimcovmat = {}

                    for choice in choices :
                        if choice=='C' :
                            stimtreatment='whiten'
                        elif choice=='F' :
                            stimtreatment='whiten'
                        elif choice=='U' :
                            stimtreatment='whiten'

                        stimtreatment = 'none'

                        STC_eigs[choice], STC_covmats[choice], W[choice], Nstimcovmat[choice] = Build_STC(Spike_Tables[choice],stim,stimtype, STC_on, choice, numsamples, t_before, t_after, stimtreatment, cutoff)
                #--------------------------------------------------------------------------------
                #Plot STCs
                #Eigenvalues
                if ploteo[4]==1:
                    for choice in choices :
                        display_STC_eigs(STC_eigs[choice], Counts_Tables[choice], STC_on, t_before*200, choice, dire,titles,group)
                #Eigenvectors
                filters = 8    
                if ploteo[5]==1:
                    display_STC(STC_eigs, Counts_Tables, STC_on, filters, t_before*200, choices, dire,PW,titles,group)    

23
m1s1


/home/matias/miniconda/lib/python3.4/site-packages/ipykernel/__main__.py:131: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/home/matias/miniconda/lib/python3.4/site-packages/ipykernel/__main__.py:132: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


ValueError: could not broadcast input array from shape (3) into shape (4)

In [17]:
#STC_on[255]
#PW[55]
np.reshape(ResponseRC[777],[5,5])

array([[ 0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.],
       [ 0.,  0., -1.,  0.,  0.],
       [-1.,  1., -1.,  1.,  0.],
       [ 0.,  0.,  0.,  0.,  0.]])

In [ ]:
#test

#Spikes file from klusterkwik
sp_file = 'MEAS-150716-3_ele17_ele24.kwik'
#Spikes, sampling_freq = readkwikinfo(sp_file)
model = KwikModel(sp_file)

model.describe()

In [ ]:
t_before = .005
t_after = .045
PSTH_spike_counts_up, PSTH_spike_counts_down, hist_inds_up, hist_inds_down, trials = BuildPSTHdual(16, Spikes[78].spike_times, Vtag1)

In [ ]:
display_conditional_PSTHs(78, 16, hist_inds_up, hist_inds_down, PSTH_spike_counts_up, PSTH_spike_counts_down, trials, './PDFS/PDFpsth/', t_before, t_after)

In [ ]:
t_before = .005
t_after = .045
PSTH_spike_counts_up, PSTH_spike_counts_down, hist_inds_up, hist_inds_down, trials = BuildPSTHdual(12, Spikes[78].spike_times, Vtag1)

In [ ]:
display_conditional_PSTHs(78, 12, hist_inds_up, hist_inds_down, PSTH_spike_counts_up, PSTH_spike_counts_down, trials, './PDFS/PDFpsth/', t_before, t_after)